In [ ]:
pip install opencv-python

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [6]:
# Cargar el video grabado
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

In [7]:
# Cargar el modelo pre-entrenado para la detección de personas
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Definir las coordenadas y el tamaño del ROI (región de interés)
roi_x = 115  # Coordenada x del esquina superior izquierda del ROI
roi_y = 90  # Coordenada y del esquina superior izquierda del ROI
roi_width = 350  # Ancho del ROI
roi_height = 190  # Altura del ROI
frames_free = 0
frames_occupied = 0
# Redefinir el tamaño del video para reducir la resolución (ajusta según sea necesario)
frame_width = 640
frame_height = 360
out = cv2.VideoWriter('output_video_two.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Redimensionar el cuadro para reducir la resolución
    frame = cv2.resize(frame, (frame_width, frame_height))

    # Obtener la región de interés (ROI) del cuadro actual
    roi = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

    # Detectar personas en la región de interés
    boxes, _ = hog.detectMultiScale(roi)

    # Determinar si hay personas en la región de interés
    if len(boxes) > 0:
        # Si hay personas, marcar el ROI en rojo
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)
        frames_occupied += 1
    else:
        # Si está libre, marcar el ROI en verde
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 255, 0), 2)
        frames_free += 1

    # Escribir el cuadro en el video de salida

    #añadiendo textos de cuantos frames libres y ocupados.
    text = f'Frames Libres: {frames_free}'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.68
    font_color = (0, 255, 0)  # Color verde
    thickness = 1
    x, y = 50, 306 # Coordinates to place the text
    cv2.putText(frame, text, (x, y), font, font_scale, font_color, thickness)

    text = f'Frames Ocupados: {frames_occupied}'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.68
    font_color = (0, 0, 255)  # Color rojo
    thickness = 1
    x, y = 50, 332 # Coordinates to place the text
    cv2.putText(frame, text, (x, y), font, font_scale, font_color, thickness)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()